In [ ]:
# Import all libraries
import rasterio
from rasterio.plot import show
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from glob import glob
import os
import sys

### This sample code is based on Scikit-learn K-means classification

https://scikit-learn.org/stable/modules/clustering.html#k-means

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

#### This code has been tested with Landsat 8 OLI, Landsat 7 ETM+, Landsat 4 TM, and Sentinel 2 MSI data

In [ ]:
### open and read input raster file
ras = 'D:/your_input_directory/your_input_raster.tif'

In [ ]:
### read opened raster as numpy array
with rasterio.open(ras, 'r')  as src:
    show(src1, cmap='viridis') # to visualize the raster
    ras_arr = src.read(1)      # to read raster as numpy array
### these steps are not mendatory
### but can run to see the info of input raster
    print(ras_arr.shape)       # to see the number of rows and columns
    print(ras_arr)             # to print the array
    print(np.min(ras_arr))     # to print minimum value of the array
    print(np.max(ras_arr))     # to print maximum value of the array

In [ ]:
### Step 1: flattening or reshape the 2D numpy array into 1D numpy array
### with (-1,1) shape, which means we will have (rows, 1) shape, many rows and 1 column
### for ML always we need flatten 2D numpy array into rows and 1 column
ras_flat = ras_arr.reshape([-1, 1]) 
print(ras_flat.shape)
print(ras_flat)
print(np.min(ras_flat))
print(np.max(ras_flat))
### check if there are any Null, infinate, or unexpected
### values in the data set
print(np.any(np.isnan(ras_flat))) #Answer must be 'False'
print(np.all(np.isfinite(ras_flat))) #Answer must be 'True'
### if there are no error values then proceed Step 2
### otherwise, proceeed Step 1.1

In [ ]:
### Step 1.1: clean the data set
### this will replace all error values 

ras_flat = np.nan_to_num(ras_flat, nan=0, posinf=0, neginf=0)
print(ras_flat.shape)
print(ras_flat)
print(np.min(ras_flat))
print(np.max(ras_flat))
### check if there are any Null, infinate, or unexpected
### values in the data set
print(np.any(np.isnan(ras_flat))) #Answer must be 'False'
print(np.all(np.isfinite(ras_flat))) #Answer must be 'True'

In [ ]:
### Step 2: k-mean with SciKit learn
from sklearn.cluster import KMeans
ras_km = KMeans(n_clusters=6, random_state=0).fit(ras_flat) 
### Number of cluster is 6 for this test case, 
### but it can be any number, for more details look Scikit-learn website 
### clustering kmeans data (1D numpy array)
ras_km_cluster = ras_km.labels_
print(ras_km_cluster.shape)
print(ras_km_cluster)
print(np.min(ras_km_cluster))
print(np.max(ras_km_cluster))
### reshape kmneas 1D cluster data into 2D numpy array
ras_km_data = ras_km_cluster.reshape(ras_arr.shape)
### plot array
plt.imshow(ras_km_data, cmap="Paired") # you can use any cmap from matplotlib cmap list
### plot colorbar
plt.colorbar(shrink=0.5)
plt.clim(np.min(ras_km_data), np.max(ras_km_data))
print(ras_km_data.shape)
print(ras_km_data)
print(np.min(ras_km_data))
print(np.max(ras_km_data))

In [ ]:
### Step 3: save the output as jpg file
### make the output file directory
fig_direc = 'D:/your_output_directory/CI'
if not os.path.exists(fig_direc):
    os.makedirs(fig_direc)
### Save the figure as jpg format
plt.savefig('D:/your_output_directory/your_output_raster.jpg', dpi=600) # dpi can be changed according to 
# needs but 600 is high enough and such good resolution

In [ ]:
### Step 4: save the output as GeoTiff file
### Get metadata from input raster
with rasterio.open(ras) as src:
    meta = src.meta
meta.update(dtype=rasterio.float32)
### Create output folder and write output file in it as geotiff
file = "D:/your_output_directory/your_output_raster.tif"
os.makedirs(os.path.dirname(file), exist_ok=True)
with rasterio.open(file, 'w', **meta) as dst:
    dst.write(ras_km_data.astype(rasterio.float32), 1)